In [32]:
import pandas as pd
import numpy as np
from sklearn import linear_model

from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()

from geopy.geocoders import Nominatim
geolocator = Nominatim()

import geopy.distance

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Read in the csv file
freight_df = pd.read_csv('Freight Regression Analysis 2017 Data.csv')
freight_df = freight_df[['Origin Zip','Destination Zip','Pieces','Weight','Commodity Class','Base Retail']]
freight_df.head()
freight_df.tail()

,Origin Zip,Destination Zip,Pieces,Weight,Commodity Class,Base Retail
0,75261,77303,1,2000,60.0,176.21
1,75261,29605,1,447,60.0,155.11
2,75261,76048,1,331,60.0,130.80
3,75261,76048,1,331,60.0,130.80
4,75261,75028,1,1236,60.0,145.00


,Origin Zip,Destination Zip,Pieces,Weight,Commodity Class,Base Retail
582,75261,83716,7,8885,60.0,1712.17
583,75261,27616,12,18785,65.0,2114.14
584,90660,75261,12,26480,60.0,2140.63
585,75261,27616,11,11569,60.0,2210.27
586,75261,V5C5V1,4,5181,60.0,8921.20


In [18]:
freight_df.dtypes
freight_df.shape

Origin Zip           int64
Destination Zip     object
Pieces               int64
Weight               int64
Commodity Class    float64
Base Retail        float64
origin_lat         float64
origin_lng         float64
destination_lat    float64
destination_lng    float64
dtype: object

(587, 10)

# Preprocess the data

In [5]:
# Get origin zip codes as a list
origin_zips = freight_df['Origin Zip'].tolist()

origin_coord_list = []
for i in origin_zips:    
    zip_code = search.by_zipcode(i)
    zip_lat = zip_code.Latitude
    zip_lng = zip_code.Longitude 
    zip_coord = (zip_lat,zip_lng)
    origin_coord_list.append(zip_coord)
    
origin_coord_list    

[(None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, None),
 (None, No

In [10]:
# Convert origin list of latitude and longtitude to a dataframe
origin_coord_list_df = pd.DataFrame(origin_coord_list )
origin_coord_list_df.columns = ['origin_lat', 'origin_lng']

In [11]:
# Get destination zip codes as a list
destination_zips = freight_df['Destination Zip'].tolist()

destination_coord_list = []
for i in destination_zips:     
    zip_code = search.by_zipcode(i)
    zip_lat = zip_code.Latitude
    zip_lng = zip_code.Longitude 
    zip_coord = (zip_lat,zip_lng)
    destination_coord_list.append(zip_coord)
    
destination_coord_list

[(30.38273019999999, -95.38348540000001),
 (34.7671945, -82.37773349999998),
 (32.4084693, -97.80688509999999),
 (32.4084693, -97.80688509999999),
 (33.0478852, -97.0605998),
 (32.9464268, -97.07135720000001),
 (25.4308762, -80.3992652),
 (26.7712632, -80.0702555),
 (32.9464268, -97.07135720000001),
 (32.9464268, -97.07135720000001),
 (36.4242854, -82.97166170000001),
 (32.7476492, -97.0924899),
 (36.54217379999999, -82.10606829999998),
 (40.5417839, -105.04405200000001),
 (29.858217800000002, -90.2057108),
 (36.4242854, -82.97166170000001),
 (40.5417839, -105.04405200000001),
 (32.5355472, -92.4801989),
 (41.5214385, -83.57096399999998),
 (30.3889868, -97.67108890000002),
 (27.6515548, -97.50306690000001),
 (39.09614929999999, -96.53564490000001),
 (35.9848005, -83.6135572),
 (39.7794206, -85.7256372),
 (40.5417839, -105.04405200000001),
 (32.9464268, -97.07135720000001),
 (30.160243599999998, -95.1953554),
 (34.9208336, -82.37773349999998),
 (29.198795699999998, -81.04234290000002),


In [12]:
# Convert origin list of latitude and longtitude to a dataframe
destination_coord_list_df = pd.DataFrame(destination_coord_list)
destination_coord_list_df.columns = ['destination_lat', 'destination_lng']
destination_coord_list_df.head()
destination_coord_list_df.tail()

,destination_lat,destination_lng
0,30.382730,-95.383485
1,34.767195,-82.377733
2,32.408469,-97.806885
3,32.408469,-97.806885
4,33.047885,-97.060600


,destination_lat,destination_lng
582,43.560587,-116.132825
583,35.877441,-78.554728
584,NaN,NaN
585,35.877441,-78.554728
586,NaN,NaN


In [13]:
# Concat the origin and destination coordinates to original dataframe
freight_df = pd.concat([freight_df,origin_coord_list_df,destination_coord_list_df],axis=1) 
freight_df.head()

,Origin Zip,Destination Zip,Pieces,Weight,Commodity Class,Base Retail,origin_lat,origin_lng,destination_lat,destination_lng
0,75261,77303,1,2000,60.0,176.21,NaN,NaN,30.382730,-95.383485
1,75261,29605,1,447,60.0,155.11,NaN,NaN,34.767195,-82.377733
2,75261,76048,1,331,60.0,130.80,NaN,NaN,32.408469,-97.806885
3,75261,76048,1,331,60.0,130.80,NaN,NaN,32.408469,-97.806885
4,75261,75028,1,1236,60.0,145.00,NaN,NaN,33.047885,-97.060600


In [15]:
# Unique Origin Zipcodes: 75261, 90630, 32218, 90660, 92649, 92618, 90814, 91733, 29334
# only 75261 doesn't have coordinates [Used code below to check each of the following values]
# freight_df[(freight_df['Origin Zip'] == 29334)]

# fill the 75261 zip codes with a (lat, long) of 32.8981, -97.0337

freight_df['origin_lat'].fillna(value=32.8981, inplace=True)
freight_df['origin_lng'].fillna(value=-97.0337, inplace=True)
freight_df.head()


,Origin Zip,Destination Zip,Pieces,Weight,Commodity Class,Base Retail,origin_lat,origin_lng,destination_lat,destination_lng
0,75261,77303,1,2000,60.0,176.21,32.8981,-97.0337,30.382730,-95.383485
1,75261,29605,1,447,60.0,155.11,32.8981,-97.0337,34.767195,-82.377733
2,75261,76048,1,331,60.0,130.80,32.8981,-97.0337,32.408469,-97.806885
3,75261,76048,1,331,60.0,130.80,32.8981,-97.0337,32.408469,-97.806885
4,75261,75028,1,1236,60.0,145.00,32.8981,-97.0337,33.047885,-97.060600


In [19]:
freight_null = freight_df[freight_df['destination_lat'].isnull()]
freight_null.shape
freight_null.head()

(32, 10)

,Origin Zip,Destination Zip,Pieces,Weight,Commodity Class,Base Retail,origin_lat,origin_lng,destination_lat,destination_lng
74,90630,53547,1,132,60.0,162.38,33.816539,-118.036174,NaN,NaN
77,32218,75261,1,2063,60.0,136.85,30.468683,-81.669253,NaN,NaN
78,90630,75261,1,671,60.0,159.13,33.816539,-118.036174,NaN,NaN
84,75261,L3P2L8,1,206,60.0,172.50,32.898100,-97.033700,NaN,NaN
89,90630,75261,1,384,65.0,152.68,33.816539,-118.036174,NaN,NaN


In [20]:
#drop NaN rows
freight_df = freight_df.dropna()

In [22]:
freight_df["Origin coords"] = list(zip(freight_df["origin_lat"], freight_df["origin_lng"]))
freight_df["Destination coords"] = list(zip(freight_df["destination_lat"], freight_df["destination_lng"]))

/Users/patrickdancel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/patrickdancel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
# create a dataframe with the ''Pieces', 'Weight', 'Commodity Class','Base Retail', 'Origin coords', 'Destination coords' columns

final_freight_df = freight_df[['Pieces', 'Weight', 'Commodity Class','Base Retail', 'Origin coords', 'Destination coords']]
final_freight_df.head()

,Pieces,Weight,Commodity Class,Base Retail,Origin coords,Destination coords
0,1,2000,60.0,176.21,"(32.8981, -97.0337)","(30.3827302, -95.3834854)"
1,1,447,60.0,155.11,"(32.8981, -97.0337)","(34.7671945, -82.3777335)"
2,1,331,60.0,130.80,"(32.8981, -97.0337)","(32.4084693, -97.8068851)"
3,1,331,60.0,130.80,"(32.8981, -97.0337)","(32.4084693, -97.8068851)"
4,1,1236,60.0,145.00,"(32.8981, -97.0337)","(33.0478852, -97.0605998)"


In [29]:
# Calculate the distance between the origin and destination coordinates in Miles
distance_list = []
for item in final_freight_df.itertuples():
    coords_1 = item[5]
    coords_2 = item[6]
    distance = geopy.distance.vincenty(coords_1, coords_2).miles
    distance_list.append(distance)
final_freight_df['distance(mi)'] = distance_list

final_freight_df.head()
final_freight_df.tail()

/Users/patrickdancel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Pieces,Weight,Commodity Class,Base Retail,Origin coords,Destination coords,distance(mi)
0,1,2000,60.0,176.21,"(32.8981, -97.0337)","(30.3827302, -95.3834854)",198.726256
1,1,447,60.0,155.11,"(32.8981, -97.0337)","(34.7671945, -82.3777335)",851.967779
2,1,331,60.0,130.80,"(32.8981, -97.0337)","(32.4084693, -97.8068851)",56.302059
3,1,331,60.0,130.80,"(32.8981, -97.0337)","(32.4084693, -97.8068851)",56.302059
4,1,1236,60.0,145.00,"(32.8981, -97.0337)","(33.0478852, -97.0605998)",10.439673


,Pieces,Weight,Commodity Class,Base Retail,Origin coords,Destination coords,distance(mi)
580,15,18929,65.0,1619.13,"(32.8981, -97.0337)","(35.8774409, -78.5547283)",1074.004985
581,12,18946,60.0,1672.52,"(32.8981, -97.0337)","(35.8774409, -78.5547283)",1074.004985
582,7,8885,60.0,1712.17,"(32.8981, -97.0337)","(43.5605869, -116.1328252)",1267.990337
583,12,18785,65.0,2114.14,"(32.8981, -97.0337)","(35.8774409, -78.5547283)",1074.004985
585,11,11569,60.0,2210.27,"(32.8981, -97.0337)","(35.8774409, -78.5547283)",1074.004985


In [34]:
final_freight_df.columns

Index(['Pieces', 'Weight', 'Commodity Class', 'Base Retail', 'Origin coords',
       'Destination coords', 'distance(mi)'],
      dtype='object')

In [35]:
# Select the five columns to run the linear regression model
final_freight_df = final_freight_df[['Pieces', 'Weight', 'Commodity Class', 'Base Retail', 'distance(mi)']]
final_freight_df.head()
# Save to a csv file
final_freight_df.to_csv('freight_data_clean.csv')

,Pieces,Weight,Commodity Class,Base Retail,distance(mi)
0,1,2000,60.0,176.21,198.726256
1,1,447,60.0,155.11,851.967779
2,1,331,60.0,130.80,56.302059
3,1,331,60.0,130.80,56.302059
4,1,1236,60.0,145.00,10.439673
